In [ ]:
#!pip3 install -U spacy
#!python -m spacy download en_core_web_md

In [11]:
import json

with open('ner/ner_train.json', mode='r', encoding='utf-8') as fd:
    ner_train_data = json.loads(fd.read());

print(ner_train_data[0])
print(ner_train_data[5])

{'id': 1, 'annotations': [{'id': 1, 'completed_by': 1, 'result': [], 'was_cancelled': False, 'ground_truth': False, 'created_at': '2024-03-02T10:51:06.916485Z', 'updated_at': '2024-03-02T10:51:06.916501Z', 'draft_created_at': None, 'lead_time': 10.919, 'prediction': {}, 'result_count': 0, 'unique_id': 'e56fa1cb-9a1e-4685-89cf-ccb2437e4df8', 'import_id': None, 'last_action': None, 'task': 1, 'project': 2, 'updated_by': 1, 'parent_prediction': None, 'parent_annotation': None, 'last_created_by': None}], 'file_upload': '70091d08-sample.txt', 'drafts': [], 'predictions': [], 'data': {'text': '@fansoniclove Gold the Tenrec'}, 'meta': {}, 'created_at': '2024-03-02T10:50:00.501337Z', 'updated_at': '2024-03-02T10:51:06.935188Z', 'inner_id': 1, 'total_annotations': 1, 'cancelled_annotations': 0, 'total_predictions': 0, 'comment_count': 0, 'unresolved_comment_count': 0, 'last_comment_updated_at': None, 'project': 2, 'updated_by': 1, 'comment_authors': []}
{'id': 6, 'annotations': [{'id': 6, 'comp

In [12]:
print(ner_train_data[16])

{'id': 17, 'annotations': [{'id': 17, 'completed_by': 1, 'result': [{'value': {'start': 0, 'end': 4, 'text': 'Gold', 'labels': ['COM']}, 'id': 'j7D1UyehxE', 'from_name': 'label', 'to_name': 'text', 'type': 'labels', 'origin': 'manual'}], 'was_cancelled': False, 'ground_truth': False, 'created_at': '2024-03-02T11:09:57.820429Z', 'updated_at': '2024-03-03T02:00:26.895744Z', 'draft_created_at': '2024-03-02T11:09:50.008467Z', 'lead_time': 188.464, 'prediction': {}, 'result_count': 0, 'unique_id': 'a9387a03-9297-4734-831a-574ddb42bcc6', 'import_id': None, 'last_action': None, 'task': 17, 'project': 2, 'updated_by': 1, 'parent_prediction': None, 'parent_annotation': None, 'last_created_by': None}], 'file_upload': '70091d08-sample.txt', 'drafts': [], 'predictions': [], 'data': {'text': 'Gold Price News and Forecast: XAU/USD is trapped in daily support and resistance. Posted by: EUR Editor  in EUR  1 min ago  XAU/USD struggles below $1,800 amid risk-off mood. Gold steps back from intraday high

In [13]:
import spacy
from spacy.tokens import DocBin

nlp = spacy.load("en_core_web_sm")

db = DocBin()
for item in ner_train_data:
    text = item['data']['text']
    annotations = item['annotations']

    if len(text) < 20:
        continue

    doc = nlp(text)
    ents = []

    # [E1010] Unable to set entity information for token 0 which is included in more than one span in entities, blocked, missing or outside.
    # Eg:
    #   text: Gold Price News and Forecast: XAU/USD is trapped in daily support and resistance. Posted by: EUR Editor  in EUR  1 min ago  XAU/USD struggles below $1,800 amid risk-off mood. Gold steps back from intraday high while flashing $1,772 as a quote amid Friday’s Asian session. In doing … Read Full Story at source (may require registration)     Latest posts by EUR Editor ( see all )
    #   ents: [Gold Price News, XAU/USD, daily, 1 min ago, XAU/USD, 1,800, 1,772, Friday, Asian, … Read Full Story, EUR Editor, Gold]
    #   span as text[0:16] and span as text[0:4] are overlaping each other -> token 0 which is included in more than one span

    annotated = annotations[0]

    for r in annotated['result']:
        value = r['value']
        span = doc.char_span(
            value['start'],
            value['end'],
            label=value['labels'][0]
            )
        if span is None:
            print('[result] ', r);
            print('[span] ', span);
            print('[text] ', item['data']['text']);
            raise Error('The entity is marked inproperly that makes span to None');
        else:
            ents.append(span)

    for ent in doc.ents:
        span = doc.char_span(ent.start_char, ent.end_char, label=ent.label_)
        is_span_overlaping = False
        for included_span in ents:
            if ((included_span.start <= span.start and span.start <= included_span.end)
                or (included_span.start <= span.end and span.end <= included_span.end)
                or (span.start <= included_span.start and included_span.start <= span.end)
                or (span.start <= included_span.end and included_span.end <= span.end)):
                    is_span_overlaping = True

        if is_span_overlaping:
            continue

        ents.append(span)

    if len(ents) == 0:
        continue
    doc.ents = ents
    db.add(doc)

db.to_disk("./ner/train.spacy")

In [16]:
text = "Gold Rate Today, 30 April 2021: Gold, Silver fall, know – what are the 10 grams gold rate today https://t.co/kSVOTVuXw5"


In [17]:
doc = nlp(text)
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Gold 0 4 COM
Today 10 15 DATE
30 April 2021 17 30 DATE
10 grams 71 79 QUANTITY


In [18]:
text = "Tbvh, Adekunle Gold really did magic on this song he issued out titled #IIWII .. Man always stepping up his game.. https://t.co/LEDEq0AdF6"
doc = nlp(text)
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Adekunle Gold 6 19 ORG


In [19]:
text = "Did and Dig!. Gold is one of the most desired and useful metals in the world. Not only could it be beautifully shaped and sculpted, the precious yellow metal conducts electricity and does not tarnish. This article could possibly give you more knowledge about the uses of the precious metal gold. Check it here!   These qualities could make it the metal of choice for the industrial, medical and technology businesses. Arguably no other metal has been said to have a record throughout history, with almost every established culture using gold to symbolise power, beauty, purity and accomplishment.    Today, gold still seems to occupy an important place important place in our culture and society people could posibly use it to make our one of the most prized objects: wedding rings, Olympic medals, money, jewellery, Oscars, Grammys, crucifixes, art and many more. This is a sponsored post. Check disclaimer on profile and landing page"
doc = nlp(text)
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Dig 8 11 PERSON
Today 600 605 DATE
Olympic 783 790 DATE
Oscars 817 823 ORG
Grammys 825 832 GPE


In [20]:
text = "Gold Forecast: Hovering Between Moving Averages. Pay close attention to the rate of change in the yield of bonds, because if it is a slow and gradual rise, then I believe that gold should do fairly well. Gold markets initially tried to rally during the course of the trading session on Thursday but failed to continue going higher at the 200 day EMA as it continues to be quite resistive. At this point, the market then felt to reach down below the 50 day EMA, which of course was a very negative sign. However, by the end of the session we turned around to break above the 50 day EMA and now it looks like the market is essentially doing the same thing it has been doing for several days. These are two major averages that a lot of people pay attention to, so it should not be overly surprising that we are stuck in this general vicinity. Furthermore, you have to pay close attention to the US dollar, because the gold market is highly influenced by it. Now that that we have broken down below the 50 day EMA, it looks like the market is finding value hunters underneath just above the $1750 level. I think what we are looking at here is a market that is trying to figure out where we are getting ready to go longer term. When you look at the charts you can see that we have formed a double bottom just below the $1700 level, so I think if we break down below the $1750 level, then we will go testing that area. However, the $1750 level above had been resisted previously, broken out above, and now has been retested. At this point, we have to question whether or not this attempt to form a basing pattern has started to stick? A break above the $1800 level opens up the possibility of a much bigger move, and therefore I think that is the key for buying this market. Until then, I would be hesitant to put a lot of money into the gold market, but it certainly looks as if we are trying to turn things around. It could be very noisy over the next couple of weeks, but I do have my eye on this market as a confirmation of the return could very well send this market looking towards the highs again. Pay close attention to the rate of change in the yield of bonds, because if it is a slow and gradual rise, then I believe that gold should do fairly well"
doc = nlp(text)
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Between Moving Averages 24 47 LOC
Thursday 286 294 DATE
200 day 338 345 DATE
EMA 346 349 ORG
the 50 day 445 455 DATE
EMA 456 459 ORG
the end 515 522 DATE
EMA 581 584 ORG
several days 676 688 DATE
two 700 703 CARDINAL
US 892 894 GPE
the 50 day 995 1005 DATE
EMA 1006 1009 ORG
1750 1088 1092 MONEY
1700 1315 1319 MONEY
1750 1366 1370 MONEY
1750 1427 1431 MONEY
1800 1648 1652 MONEY
the next couple of weeks 1939 1963 DATE


In [6]:
import json

with open('sentiment_analysis_spacy/sentiment_train.json', mode='r', encoding='utf-8') as fd:
    sent_train_data = json.loads(fd.read());

print(sent_train_data[5])

{'id': 2108, 'annotations': [{'id': 470, 'completed_by': 1, 'result': [{'value': {'choices': ['Negative']}, 'id': '8CdQ4W1i46', 'from_name': 'sentiment', 'to_name': 'text', 'type': 'choices', 'origin': 'manual'}], 'was_cancelled': False, 'ground_truth': False, 'created_at': '2024-03-03T08:17:06.304621Z', 'updated_at': '2024-03-03T08:24:43.405397Z', 'draft_created_at': None, 'lead_time': 97.557, 'prediction': {}, 'result_count': 0, 'unique_id': '14b4b0a3-db9d-4c72-936c-f9c8df8be723', 'import_id': 238, 'last_action': None, 'task': 2108, 'project': 6, 'updated_by': 1, 'parent_prediction': None, 'parent_annotation': None, 'last_created_by': None}], 'file_upload': '9a09ceda-sentiment_train.json', 'drafts': [], 'predictions': [], 'data': {'text': 'Gold edges lower as firmer US yields pinch appeal https://t.co/P0HPkmXvBZ'}, 'meta': {}, 'created_at': '2024-03-03T08:17:06.279707Z', 'updated_at': '2024-03-03T08:24:43.432821Z', 'inner_id': 6, 'total_annotations': 1, 'cancelled_annotations': 0, 't

In [63]:
from pathlib import Path
import spacy
from spacy.tokens import DocBin

def read_categories():
    return Path('sentiment_analysis_spacy/categories.txt').open().read().strip().split("\n")

categories = read_categories();
print(categories)
nlp = spacy.blank("en")
db = DocBin()
for item in sent_train_data:
    text = item['data']['text']
    annotated = item['annotations'][0]

    if len(text) < 20:
        continue

    result = annotated['result']
    if len(result) == 0:
        continue;
    label = result[0]['value']['choices'][0]

    doc = nlp.make_doc(text)
    doc.cats = {category: 0 for category in categories}
    # True labels get value 1
    doc.cats[label] = 1

    print(doc)
    db.add(doc)

db.to_disk("./sentiment_analysis_spacy/train.spacy")

['Positive', 'Negative', 'Neutral']
"US 500 Futures Discussions. Hey guys what is gold's history in May?  
Good time to invest in gold?"
Gold edges lower as firmer US yields pinch appeal https://t.co/P0HPkmXvBZ
Gold Futures Discussions. So, any more short paper expire? Moy said, but it's only a matter of time before the short contracts keeping the price down expire.
APX 1.75  Overbought (RSI)  Trade range 1.40 -1.80  Gold falls all eyes on FED rate decision https://t.co/BgU5kmqPX2
Gold Futures Discussions. I read somewhere that ounces of gold per person were almost constant in history, but not the money...  all previous reserve currencies failed, probably it is dollar turn with Modern Monetary theory at WH
Gold Price News and Forecast: XAU/USD is trapped in daily support and resistance. Posted by: EUR Editor  in EUR  1 min ago  XAU/USD struggles below $1,800 amid risk-off mood. Gold steps back from intraday high while flashing $1,772 as a quote amid Friday’s Asian session. In doing … R

In [37]:
# number of row has been used for training
padding = 231
with open('sample.txt', mode='r', encoding='utf-8') as fd:
    samples = fd.readlines()
samples = samples[padding:]

In [38]:
import spacy
ner = spacy.load("ner/output/model-last")
# for doc in nlp.pipe(texts, disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"]):
#   print([(ent.text, ent.label_) for ent in doc.ents])

In [40]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline
from datasets import Dataset, DatasetDict

model = BertForSequenceClassification.from_pretrained("ahmedrachid/FinancialBERT-Sentiment-Analysis",num_labels=3)
tokenizer = BertTokenizer.from_pretrained("ahmedrachid/FinancialBERT-Sentiment-Analysis")

bert_sent = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)

sentences = ["Operating profit rose to EUR 13.1 mn from EUR 8.7 mn in the corresponding period in 2007 representing 7.7 % of net sales.",  
             "Bids or offers include at least 1,000 shares and the value of the shares must correspond to at least EUR 4,000.", 
             "Raute reported a loss per share of EUR 0.86 for the first half of 2009 , against EPS of EUR 0.74 in the corresponding period of 2008.", 
             ]

#results = bert_sent(sentences)
#print(results)

In [41]:
gold_silver_com_sample = []
com_label = 'COM'
for sample in samples:
    doc = ner(sample)
    for ent in doc.ents:
        if ent.label_ == com_label and ent.text.lower() in ['gold', 'silver']:
            # print(sample);
            # print(ent.text, ent.start_char, ent.end_char, ent.label_)
            gold_silver_com_sample.append(sample);

In [60]:
import spacy
sent = spacy.load("sentiment_analysis_spacy/output/model-last")

text_length = 1000
headers = ['Sample', 'label by Spacy', 'label by FinancialBERT', 'Comparison']
data = []
for sample in gold_silver_com_sample:

    doc = sent(sample)
    spacy_label = max(doc.cats.items(), key=lambda x: x[1])[0]
    spacy_label = spacy_label.lower()

    # model's max_seq_length is 512, FinancialBERT is not able to process too long documents
    if len(sample.split()) < 320:
        bert_predicted = bert_sent(sample)
        bert_label = bert_predicted[0]['label'].lower()
        unmatched = 'X' if spacy_label != bert_label else ''
    else:
        bert_label = 'n/a'
        unmatched = ''


    data.append([sample[:text_length], spacy_label, bert_label, unmatched])

In [61]:
# !pip3 install -U tabulate
import tabulate
table = tabulate.tabulate(
    data,
    headers=headers,
    tablefmt='simple',
    colalign=('center', 'left','center', 'center'),
    maxcolwidths=[4, 65, 8, 8], showindex="always"
    )
print(table)

     Sample                                                              label by Spacy    label by FinancialBERT   Comparison
---  -----------------------------------------------------------------  ----------------  ------------------------  ------------
 0   Silver Futures Discussions. silver 25.90                               negative              neutral           X
 1   Gold 💫 bagged, learned so much here, and nice people too              neutral               neutral
     https://t.co/g9qQ8sbDue
 2   Gold Futures Discussions. Bears game...                                negative              neutral           X
 3   Gold Rate Today, 30 April 2021: Gold, Silver fall, know – what         negative              neutral           X
     are the 10 grams gold rate today https://t.co/kSVOTVuXw5
 4   @BTC_Archive @PeterSchiff Gold price is suppressed by the              negative              neutral           X
     government. Buy Bitcoin
 5   Silver Futures Discussions. usd/inr at sup

In [12]:
# !pip3 install -U 'transformers[torch]'
# !pip3 install -U dataset
# !pip3 install -U numpy
# !pip3 install -U evaluate
# !pip install -U scikit-learn

In [2]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30873, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [3]:
for param in model.bert.embeddings.parameters():
  param.requires_grad = False

In [4]:
for param in model.bert.encoder.parameters():
  param.requires_grad = False

In [7]:
from pathlib import Path

id2label = model.config.id2label
label2id = model.config.label2id

dataset = []
for item in sent_train_data:
    text = item['data']['text']
    annotated = item['annotations'][0]

    if len(text) < 20:
        continue

    result = annotated['result']
    if len(result) == 0:
        continue;
    label = result[0]['value']['choices'][0]
    label = label.lower()

    dataset.append({
        'text': text,
        'label': label2id[label]
        })

print(len(dataset))
print(dataset[3])


45
{'text': "Gold Futures Discussions. So, any more short paper expire?\xa0Moy said, but it's only a matter of time before the short contracts keeping the price down expire.", 'label': 1}


In [8]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

dataset = Dataset.from_list(dataset)
tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map: 100%|██████████████████████████████████████████████████████████████████████████████████████████████| 45/45 [00:00<00:00, 671.80 examples/s]


In [18]:
from transformers import TrainingArguments, Trainer
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    return metric.compute(predictions=predictions, references=labels)

batch_size = 10
training_args = TrainingArguments(
  output_dir="output_trainer",
  num_train_epochs=20,
  per_device_train_batch_size=batch_size,
  per_device_eval_batch_size=batch_size,
  evaluation_strategy="epoch",
  save_total_limit=2,
  gradient_checkpointing=True,  #  If True, use gradient checkpointing to save memory at the expense of slower backward pass
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    eval_dataset=tokenized_datasets,
    compute_metrics=compute_metrics,
)

In [19]:
trainer.train()

/home/sysang/.local/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/sysang/.local/lib/python3.10/site-packages/torch/utils/checkpoint.py:90: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.645612,0.755556
2,No log,0.617838,0.777778
3,No log,0.599147,0.800000
4,No log,0.581885,0.800000
5,No log,0.565041,0.800000
6,No log,0.550479,0.800000
7,No log,0.537705,0.777778
8,No log,0.526473,0.777778
9,No log,0.515407,0.777778
10,No log,0.505374,0.777778


/home/sysang/.local/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/sysang/.local/lib/python3.10/site-packages/torch/utils/checkpoint.py:90: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/home/sysang/.local/lib/python3.10/site-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentr

TrainOutput(global_step=100, training_loss=0.5484979248046875, metrics={'train_runtime': 36.6389, 'train_samples_per_second': 24.564, 'train_steps_per_second': 2.729, 'total_flos': 236802075955200.0, 'train_loss': 0.5484979248046875, 'epoch': 20.0})